# Kaggle-Tutorial

## Geospatial Analysis

---

## 인터랙티브Interactive 지도

인터랙티브 히트맵, 등치choropleth 맵 등을 만드는 방법을 알라보자!

원본: [Interactive Maps](https://www.kaggle.com/code/alexisbcook/interactive-maps)
---
### 소개

이 튜토리얼에서는 **folium** 패키지를 사용하여 인터랙티브 맵을 만드는 방법을 배울 것이다. 그 과정에서 새로운 기술을 적용하여 보스턴 범죄 데이터를 시각화 하게 될 것이다.


![](https://i.imgur.com/v6ZUGgI.png)

In [1]:
import pandas as pd
import geopandas as gpd
import math

import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

/Users/sangjunkim/opt/anaconda3/envs/Kaggle-Tutorial-Geospatial-Analysis/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### 당신의 첫 인터랙티브 지도

`folium.Map()`로 비교적 간단한 지도를 만드는 것부터 시작한다

In [2]:
# Create a map
m_1 = folium.Map(location=[42.32,-71.0589], tiles='openstreetmap', zoom_start=10)

# Display the map
m_1

몇 가지 인수argument는 지도의 모양을 커스터마이즈customize 한다.

* `location`은 지도의 초기 중심을 설정한다. 우리는 보스턴 시의 위도latitude (42.32° N)와 경도longitude (-71.0589° E)를 사용한다.
* `tiles`는 지도의 스타일을 변경한다. 이 경우에는 [OpenStreetMap](https://www.openstreetmap.org/#map=10/42.32/-71.0589) 스타일을 사용한다. 궁금하다면 [여기](https://github.com/python-visualization/folium/tree/master/folium/templates/tiles)에서 다른 옵션들의 리스트를 볼 수 있다.
* `zoom_start`는 지도의 초기 확대/축소 수준을 설명하며, 값이 높을수록 지도에 더 가깝게 확대된다.

시간을 내어 지도를 확대 및 축소, 다른 방향으로 끌어 탐색해보자.

### 자료 The data
이제 지도에 범죄 데이터를 추가해볼 것이다!

우리는 데이터 로딩에 있어서는 집중하지 않을 것이다. 대신 pandas DataFrame `범죄` 데이터가 이이 있다고 생각해자. 처음 데이터의 5행은 아래와 같다.

In [3]:
# Load the data
crimes = pd.read_csv("./input/geospatial-learn-course-data/crimes-in-boston/crimes-in-boston/crime.csv", encoding='latin-1')

# Drop rows with missing locations
crimes.dropna(subset=['Lat', 'Long', 'DISTRICT'], inplace=True)

# Focus on major crimes in 2018
crimes = crimes[crimes.OFFENSE_CODE_GROUP.isin([
    'Larceny', 'Auto Theft', 'Robbery', 'Larceny From Motor Vehicle', 'Residential Burglary',
    'Simple Assault', 'Harassment', 'Ballistics', 'Aggravated Assault', 'Other Burglary',
    'Arson', 'Commercial Burglary', 'HOME INVASION', 'Homicide', 'Criminal Harassment',
    'Manslaughter'])]
crimes = crimes[crimes.YEAR>=2018]

# Print the first five rows of the table
crimes.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
6,I182070933,724,Auto Theft,AUTO THEFT,B2,330,NaN,2018-09-03 21:25:00,2018,9,Monday,21,Part One,NORMANDY ST,42.306072,-71.082733,"(42.30607218, -71.08273260)"
8,I182070931,301,Robbery,ROBBERY - STREET,C6,177,NaN,2018-09-03 20:48:00,2018,9,Monday,20,Part One,MASSACHUSETTS AVE,42.331521,-71.070853,"(42.33152148, -71.07085307)"
19,I182070915,614,Larceny From Motor Vehicle,LARCENY THEFT FROM MV - NON-ACCESSORY,B2,181,NaN,2018-09-02 18:00:00,2018,9,Sunday,18,Part One,SHIRLEY ST,42.325695,-71.068168,"(42.32569490, -71.06816778)"
24,I182070908,522,Residential Burglary,BURGLARY - RESIDENTIAL - NO FORCE,B2,911,NaN,2018-09-03 18:38:00,2018,9,Monday,18,Part One,ANNUNCIATION RD,42.335062,-71.093168,"(42.33506218, -71.09316781)"


### 플로팅 포인트plotting point

지도에 표시해야 하는 데이터의 양을 줄이기 위해 (일시적으로) 주간 강도 사건에 국한할 것이다.

In [4]:
daytime_robberies = crimes[((crimes.OFFENSE_CODE_GROUP == 'Robbery') & \
                            (crimes.HOUR.isin(range(9,18))))]

#### folium.Maker
`folium.Maker()`를 사용하여 지도에 마커를 추가한다. 아래의 각 마커는 서로 다른 각각의 강도를 가리킨다.

In [5]:
# Create a map
m_2 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
for idx, row in daytime_robberies.iterrows():
    Marker([row['Lat'], row['Long']]).add_to(m_2)

# Display the map
m_2

#### folium.plugins.MarkerCluster

마커가 너무 많은 경우, `folium.plugins.MarkerCluster()`는 지도를 깔끔하게 정히라는데 도움이 될 수 있다. 각 마커는 `MarkerCluster` 객체object에 추가된다.

In [6]:
# Create the map
m_3 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
mc = MarkerCluster()
for idx, row in daytime_robberies.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        mc.add_child(Marker([row['Lat'], row['Long']]))
m_3.add_child(mc)

# Display the map
m_3

#### Bubble maps
**거품지도bubble maps**는 마커 대신 원을 사용한다. 각 원의 크기와 색상을 변경하여 위치와 다른 변수 간의
관계를 표시할 수도 있다.

`folium.Circle()`을 이용하면 반복적인 원을 사용한 거품지도를 만든다. 아래의 코드 셀에서, 강도 범죄가 9-12시에 일어난 경우 초록색, 13-17시에 일어난 경우는 빨간색이다.

In [7]:
# Create a base map
m_4 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

def color_producer(val):
    if val <= 12:
        return 'forestgreen'
    else:
        return 'darkred'

# Add a bubble map to the base map
for i in range(0,len(daytime_robberies)):
    Circle(
        location=[daytime_robberies.iloc[i]['Lat'], daytime_robberies.iloc[i]['Long']],
        radius=20,
        color=color_producer(daytime_robberies.iloc[i]['HOUR'])).add_to(m_4)

# Display the map
m_4

`folium.Circle()`은 여러 인수arguments를 취한다.

* `location`은 위도와 경로를 원의 중심으로 하는 목록이다
* `radius`는 원의 반경을 설정한다.
    * 기존의 거품지도에서, 각 원의 radius이 다양할 수 있다. 각 원의 생상을 변경하는데 사용되는 `color_producer()` 함수와 유사한 함수를 정의함으로써 이를 구연할 수 있다.
* `color`는 각 원의 색상을 설정한다.
    * `color_producer()`함수는 강도 사건의 위치에 대한 시간의 영향을 시각화하는데 사용된다.

### 히트맵Heatmaps
히트맵을 생성하기 위해 `folium.plugins.HeatMap()`을 사용한다. 이것은 도시의 여러 지역에서 범죄의 밀도를 보여준다. 빨간색 지역은 상대적으로 범죄 사건이 더 많이 일어난다.

대도시에서 예상할 수 있듯이 대부분의 범죄는 도심 근처에서 발생한다.

In [8]:
# Create a base map
m_5 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)

# Add a heatmap to the base map
HeatMap(data=crimes[['Lat', 'Long']], radius=10).add_to(m_5)

# Display the map
m_5

위의 코드 셀에서 볼 수 있듯 `folium.plugins.HeatMap()`은 한 쌍의 인수arguments를 취한다.

* `data`는 플로팅하려는 위치를 포함한 DataFrame이다.
* `radius`는 히트맵의 매끄러움을 제어한다. 값이 높을수록 히트맵이 더 매끄럽게 보입니다(즉, 간격이 더 적음).


### 등치 지도Choropleth maps
경찰서 관할에 따라 범죄가 어떻게 다른지 이해하기 위해 등치 지도를 만들수 있다.

첫 단계로 각 구역에 다른 행이 할당되고 "geometry" 열에 지리적 경계가 포함되는 GeoDataFrame을 만든다.

In [9]:
# GeoDataFrame with geographical boundaries of Boston police districts
districts_full = gpd.read_file('./input/geospatial-learn-course-data/Police_Districts/Police_Districts/Police_Districts.shp')
districts = districts_full[["DISTRICT", "geometry"]].set_index("DISTRICT")
districts.head()

CRSError: Invalid projection: epsg:4326: (Internal Proj Error: proj_create: SQLite error on SELECT name, type, coordinate_system_auth_name, coordinate_system_code, datum_auth_name, datum_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM geodetic_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name)

또한 각 지역의 범죄 수를 보여주는 `plot_dict`라는 Pandas 시리즈를 만emsek.


In [ ]:
# Number of crimes in each police district
plot_dict = crimes.DISTRICT.value_counts()
plot_dict.head()

`plot_dict`가 `지역districts`과 동일한 인덱스를 갖는 것이 매우 중요하다.-  이것으로 지리적 경계를 적절한 색상과 일치시키는 방법을 알 수 있는 코드임을 알 수 있다.

`folium.Choropleth()` 클래스를 사용하여, 등치 지도를 만들 수 있다. 아래 지도가 표시되지 않으면 [다른 웹 브라우저](https://github.com/python-visualization/folium/issues/812)로 시도해 볼 것.

In [ ]:
# Create a base map
m_6 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)

# Add a choropleth map to the base map
Choropleth(geo_data=districts.__geo_interface__,
           data=plot_dict,
           key_on="feature.id",
           fill_color='YlGnBu',
           legend_name='Major criminal incidents (Jan-Aug 2018)'
          ).add_to(m_6)

# Display the map
m_6


`folium.Choropleth()`는 여러 인수arguments를 취한다.

* `geo_data`는 각 지리적 영역의 경계를 포함하는 GeoJSON FeatureCollection입니다.
    * 위의 코드에서, 우리는 GeoDataFrame의 `districts`데이터를  __geo_interface__` 속성attribute이 있는 GeoJSON FeatureCollection으로 변환한다.

* `data`는 색상별로 구분할 각 지리적 영역의 값인 Pandas Series입니다.

* `key_on`은 항상 `feature.id`로 설정한다.
    * 이는 `geo_data`에 사용되는 GeoDataFrame과 `data`에서 제공하는 Pandas Series의 인덱스가 동일하다는 사실을 의미한다. 세부 사항을 이해하려면 GeoJSON Feature Collection의 구조를 더 자세히 살펴봐야 한다(여기서 "features"에 해당하는 값은 list이고, 각 항목은 "id" 키가 포함된 dictionary이다).
* `fill_color`는 색상 스케일을 설정한다.
* `legend_name`은 지도의 오른쪽 상단 모서리에 있는 범례에 레이블을 지정한다.

### 당신의 차례

일본의 추가적 지진 보강이 필요한 지역을 결정하기 위해 **[당신의 지도를 디자인](https://www.kaggle.com/kernels/fork/5832145)**하라